# Lab 3 (Due @ by 11:59 pm via Canvas/Gradescope)

Due: Tuesday Nov 7 @ 11:59 PM EST

### Submission Instructions
Submit this `ipynb` file to Gradescope (this can also be done via the assignment on Canvas).  To ensure that your submitted `ipynb` file represents your latest code, make sure to give a fresh `Kernel > Restart & Run All` just before uploading the `ipynb` file to gradescope.

### Group Work

You are encouraged to work in groups for this Lab, however each student should submit their own notebook file to Gradescope. While each Part of the Lab depends on previous parts, talking through the problem with your group should help speed up both understanding and arriving at a solution. 

In [1]:
# you might use the below modules on this lab
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
import pylab as py

## Part 1: Exploring Data and Defining a Question (20 points)

For this lab, you will use the data from our [DS 3000 Spotify Playlist](https://open.spotify.com/playlist/6Bl2siSxFI0mmdzOwy562o). A data set containing the first 100 songs is on Canvas, in the labs module, called `spotify100.csv`. This is the data set we'll be using for the entirety of the lab. In this first part, you must:

1. Read in the data and print the first few rows.
2. Choose, either on your own or via discussion with your classmates, **one numeric feature** that you are interested in predicting with **one other numeric feature**.
3. Generate a scatterplot that examines the relationship between two numeric features you chose, making sure it is labelled correctly and that the features are on the appropriate axis.
4. Discuss **in a markdown cell**, *why* you chose these features, which one you are predicting (the outcome) and which one you are predicting with (the input), **and** based on the scatterplot, your intuition as to whether a simple linear regression model will be (a) appropriate and (b) a good fit to the data.
5. Formally define, **in a markdown cell, using $\LaTeX$ typesetting** the mathematical notation for the simple linear regression model.

In [2]:
df_spotify = pd.read_csv("spotify100.csv")
df_spotify.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,duration_ms,song_title,artist_name
0,0.631,0.605,0,-8.730,1,0.0386,0.45300,0.000184,0.2910,0.250,115.281,https://api.spotify.com/v1/tracks/4LVDnRxfljiE...,295293,Pursuit Of Happiness (nightmare),Kid Cudi
1,0.531,0.766,8,-7.692,1,0.0582,0.00560,0.000000,0.2010,0.532,130.048,https://api.spotify.com/v1/tracks/1rS3J4u1G8f6...,179493,I Found A Way,Drake Bell
2,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,https://api.spotify.com/v1/tracks/4uUG5RXrOk84...,175238,I'm Good (Blue),David Guetta
3,0.620,0.712,9,-6.434,1,0.1000,0.22800,0.000003,0.0561,0.830,170.234,https://api.spotify.com/v1/tracks/4dRqYKhLVujx...,268600,This Life,Vampire Weekend
4,0.787,0.655,6,-8.339,0,0.0431,0.23800,0.000171,0.1380,0.650,102.485,https://api.spotify.com/v1/tracks/6shRGWCtBUOP...,208653,Upside Down,Jack Johnson


In [4]:
px.scatter(df_spotify, x = "energy", y = "loudness", title = "predicting loudness using energy")

I chose energy and loudness because I expected there to be a linear relationship between the two varibales. For this example, I will be predicting loudness using energy scores. Looking at the scatterplot, higher energy scores seem to have a positive correlation with higher loudness scores so it seems reasonable to fit a line through these points and that the line will be a good fit to these points.

# Part 2: Simple Linear Regression

Now, using the two features you chose in Part 1, you will implement a simple linear regression model and determine if it is adequate for predicting your chosen outcome feature with your chosen input feature.

## Part 2.1: Cross Validate, Assess Model Performance and Interpret the Full Line (40 points)

Use Leave One Out Cross Validation (LOO-CV) to fit the simple linear regression model and return the cross validated $MSE$ and $R^2$ for the model. **In a markdown cell**, interpret what the $R^2$ value represents, and discuss whether you think it is a good value. **Then**, fit the model to the full data set (i.e., do not cross validate) and return the estimated regression coefficients (intercept and slope). **In a markdown cell**, interpret what the slope and intercept mean, and discuss if their values make sense to you.

**Do all of this using `numpy` and linear algebra/math.** Do *not* use any pre-built functions.

In [11]:
def get_mse(y_true, y_pred):
    return np.mean((y_pred - y_true) ** 2)

In [22]:
x = np.array([[1] * len(df_spotify),
             df_spotify.energy]).T
y = np.array(df_spotify.loudness)

b = np.matmul(np.linalg.inv(np.matmul(x.T, x)), np.matmul(x.T, y))
yhat = np.matmul(x, b)

for obs in range(len(x)):
    trainx = np.concatenate((x[:obs, 1].reshape(-1, 1), x[obs + 1:, 1].reshape(-1, 1)))
    trainy = np.concatenate((y[:obs], y[obs + 1:]))
    
    b = np.matmul(np.linalg.inv(np.matmul(trainx.T, trainx)), np.matmul(trainx.T, trainy))

mse = get_mse(y, yhat)
r2 = 1 - (mse/y.var())
mse, r2

(5.964962484227721, 0.529492898797786)

The r2 value is the percentage of variance in the y variable that can be explained by x. The r2 value for this dataset indicated that slightly more than half of the variance in loudness can be explained by energy scores. Based on the variables we are examining, I think the r2 coefficient is a good value because knowing the energy score can help you predict loudness about half of the time.

In [18]:
b = np.matmul(np.linalg.inv(np.matmul(x.T, x)), np.matmul(x.T, y))
b

array([-16.12495899,  12.76104197])

The estimated regression coefficients are -16.12 and 12.76. This means that for the line of best fit, when there is 0 energy, there is -16.12 loudness, and that for every additional unit of energy, the loudness increases by 12.76. Looking at the above scatterplot, these values do seem to make sense as they follow the shape of the distribution.

## Part 2.2: Checking Model Diagnostics (30 points)

Use the residuals that you created in the previous part after doing LOO-CV to check the assumptions of the model. Do this using three plots:

1. A plot of the residuals in the order of the data, to check independence.
2. A plot of the residuals against your chosen $x$ feature, to check constant variance (and help re-assess linearity).
3. A normal probability (QQ) plot of the residuals, to check normality.

Then, **in a markdown cell**, provide a *detailed* discussion of if you believe the assumptions of the model are met and, either way, how you know and what that means for the utility of your model.

## Part 2.3: Summarize and Outline Future Steps (10 points)

Bring your thoughts together from all the previous parts of this lab and write a **detailed** paragraph or two of **at least five sentences** which discusses:

- Did you find that the simple linear regression model does a good job of predicting your chosen $y$ feature?
  - If so, how could this model be used practically? What were the key takeaways that you would use to summarize the results to someone interested in music?
  - If not, why isn't it an adequate model? Do you think there is still promise/things you can do to adjust the model so that it does a better job or is more adequate for predicting $y$?
- Regardless of your answer to the previous question, also reflect upon:
  - (a) If you were to do it over again, would you choose different features? Was there another question you feel you could have answered using these data?
  - (b) If you were to move forward, how would you improve upon the model you built? Whether you think it is already useful or not, remember that it could *always* be made better in some way.

I think the simple linear regression model did an adequate job of predicting the chosen y feature. I think it would have helped if there was a little more cross-validated accuracy, however the model was still sufficient in predicting loudness based on energy. The key takeaways were that there was little variance in loudness unexplained by the model and that about half of the variance in loudness could be explained by the model and the x feature, energy. This means that when looking at energy and loudness, roughly half of the correlation between the two variables can be explained by changes in energy. If I were to do the regression model over again, I would be interested at looking at the relationship between acousticness and liveness. It would be cool to see whether you could predict whether a song was performed live based on the acousticness levels on the track.